In [194]:
import pandas as pd
import numpy as np
import time
from datetime import date, datetime
import calendar
import os


# DEFINE GLOBAL VARIABLES

In [195]:
SAVE_CALCULATED_DF = True
SAVE_CALCULATED_DF_PATH_BASE_FOLDER = 'D:\ICARO\Proyectos\SeleniumSundaeSwap\Calculated'
SAVE_CALCULATED_DF_PATH_BASE_FILENAME = 'sundaeswap_calculated.csv'
SAVE_CALCULATED_SMALL_DF_PATH_BASE_FILENAME = 'sundaeswap_calculated_small.csv'
SCRAPPED_FOLDER_PATH = 'D:\ICARO\Proyectos\SeleniumSundaeSwap\Scrapped'
SCRAPPED_BASE_FILENAME = 'sundaeswap_scrapped_'
SCRAPPED_ASSET_LIST_FILE = 'sundaeswap_asset_list.csv'


# GENERATE DATAFRAME

## LOAD ASSET LIST

In [196]:
dfAssetList = pd.read_csv(SCRAPPED_FOLDER_PATH+'\\'+SCRAPPED_ASSET_LIST_FILE)
dfAssetList.head(3)

,Pair,LpFee
0,AADA/ADA,0.3
1,ADAX/ADA,0.3
2,ASHIB/ADA,1.0


## LOAD SCRAPPED FILES

In [197]:
files = os.listdir(SCRAPPED_FOLDER_PATH)
# list(files)

df = pd.DataFrame()

for file in files:
    if file.__contains__(SCRAPPED_BASE_FILENAME):
        try:
            # print('Found:', file)
            dfFound = pd.read_csv(SCRAPPED_FOLDER_PATH+'\\'+file, parse_dates=True, decimal='.')
            df = df.append(dfFound, ignore_index=True)
        except:
            print('Error loading:', file)

print('Dataframe loaded')


Dataframe loaded


In [198]:
df.head(5)

,Pair,LpFee,PairPrice,TotalAssetLocked,TotalAdaLocked,AdaVolume24hs,DateTime,Date,Time,MinuteOfDay
0,SUNDAE/ADA,0.3,0.656333,"18,959,279.47","12,443,603.15","815,402.19",2022-02-08 20:11:21.366693,2022-02-08,20:11:21,1211
1,LQ/ADA,0.3,73.940162,"116,968.87","8,648,697.29","902,727.21",2022-02-08 20:11:21.366693,2022-02-08,20:11:21,1211
2,WMT/ADA,0.3,0.668082,"8,634,990.95","5,768,885.35","653,021.69",2022-02-08 20:11:21.366693,2022-02-08,20:11:21,1211
3,PAVIA/ADA,0.3,0.106211,"11,930,680","1,267,174.93","136,408.23",2022-02-08 20:11:21.366693,2022-02-08,20:11:21,1211
4,XRAY/ADA,0.3,0.166781,"6,105,011","1,018,205.01","154,472.07",2022-02-08 20:11:21.366693,2022-02-08,20:11:21,1211


In [199]:
df.tail(1)

,Pair,LpFee,PairPrice,TotalAssetLocked,TotalAdaLocked,AdaVolume24hs,DateTime,Date,Time,MinuteOfDay
11349,cbTHC/ADA,0.3,0.046783,"201,382.97","9,421.46",NaN,2022-02-15 15:30:41.744022,2022-02-15,15:30:41,930


## CONVERT AND CALCULATE

### Convert DateTime column

In [200]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11350 entries, 0 to 11349
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Pair              11350 non-null  object        
 1   LpFee             11350 non-null  float64       
 2   PairPrice         11350 non-null  float64       
 3   TotalAssetLocked  11350 non-null  object        
 4   TotalAdaLocked    11350 non-null  object        
 5   AdaVolume24hs     9343 non-null   object        
 6   DateTime          11350 non-null  datetime64[ns]
 7   Date              11350 non-null  object        
 8   Time              11350 non-null  object        
 9   MinuteOfDay       11350 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 886.8+ KB


In [201]:
# d = date(2011, 1, 1)
# print(type(d))
# print(d.timetuple())
# unixtime = time.mktime(d.timetuple())
# unixtime

def getUnixDatetime(datetime: datetime):
    unixtime = time.mktime(datetime.timetuple())
    return '{:.0f}'.format(unixtime)

df['DateTimeUnix'] = df['DateTime'].apply(getUnixDatetime)
# https://www.unixtimestamp.com/

In [202]:
def stringToFloat(strValue):
    if (type(strValue) == float):
        # print(f'Already float {strValue}')
        return strValue
    else:
        if (strValue == '0'):
            print('found 0')
            return float(0)
        try:
            newVal = strValue.replace(',','')
            return float(newVal)
        except:
            # print(f'Error stringToFloat for {strValue}, returning 0')
            return float(0)
    
df['TotalAssetLocked'] = df['TotalAssetLocked'].fillna(0)
df['TotalAdaLocked'] = df['TotalAdaLocked'].fillna(0)
df['AdaVolume24hs'] = df['AdaVolume24hs'].fillna(0)

df['TotalAssetLocked'] = df['TotalAssetLocked'].apply(stringToFloat)
df['TotalAdaLocked'] = df['TotalAdaLocked'].apply(stringToFloat)
df['AdaVolume24hs'] = df['AdaVolume24hs'].apply(stringToFloat)

In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11350 entries, 0 to 11349
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Pair              11350 non-null  object        
 1   LpFee             11350 non-null  float64       
 2   PairPrice         11350 non-null  float64       
 3   TotalAssetLocked  11350 non-null  float64       
 4   TotalAdaLocked    11350 non-null  float64       
 5   AdaVolume24hs     11350 non-null  float64       
 6   DateTime          11350 non-null  datetime64[ns]
 7   Date              11350 non-null  object        
 8   Time              11350 non-null  object        
 9   MinuteOfDay       11350 non-null  int64         
 10  DateTimeUnix      11350 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(4)
memory usage: 975.5+ KB


### Calculate unique pair lp id

In [204]:
unique = df["Pair"] + '_'+ df["LpFee"].astype(str)
df['_PAIR_ID'] = unique
# df['_PAIR_ID'].unique()

### Calculate deltas

In [205]:
# FILTER_PAIR = 'MIN/ADA'
# FILTER_PAIR_LP_FEE = 0.3
# FILTER_PAIR_ID = FILTER_PAIR + '_'+ str(FILTER_PAIR_LP_FEE)
# filterAsset = df['_PAIR_ID'] == FILTER_PAIR_ID
# # from numpy_ext import rolling_apply
# def tryDelta(dfd: pd.DataFrame):
#     #diffs_a = pd.rolling_apply(dfd['PairPrice'], 2, lambda x: x[0] - x[1])
#     print(dfd.rolling(1)['PairPrice'])
    

# tryDelta(df[filterAsset])

In [206]:
# dif = df[filterAsset].groupby('_PAIR_ID')['PairPrice'].diff()
df['_DELTA_PAIRPRICE'] = df.groupby('_PAIR_ID')['PairPrice'].diff()
df['_DELTA_TOTALASSETLOCKED'] = df.groupby('_PAIR_ID')['TotalAssetLocked'].diff()
df['_DELTA_TOTALADALOCKED'] = df.groupby('_PAIR_ID')['TotalAdaLocked'].diff()

df['_DELTA_DATETIME_SECONDS'] = df.groupby('_PAIR_ID')['DateTime'].diff().dt.seconds
df['_DELTA_DATETIME_MINUTES'] = df.groupby('_PAIR_ID')['DateTime'].diff().dt.seconds/60

## GENERATE SMALL VERSION

In [211]:
dfsmall = df.copy()
dfsmall.drop(axis=1, columns= ['Date','Time','MinuteOfDay','LpFee','Pair','DateTime','_DELTA_DATETIME_MINUTES'], inplace=True)
dfsmall.tail(1)

,PairPrice,TotalAssetLocked,TotalAdaLocked,AdaVolume24hs,DateTimeUnix,_PAIR_ID,_DELTA_PAIRPRICE,_DELTA_TOTALASSETLOCKED,_DELTA_TOTALADALOCKED,_DELTA_DATETIME_SECONDS
11349,0.046783,201382.97,9421.46,0.0,1644949841,cbTHC/ADA_0.3,0.0,0.0,0.0,900.0


# SAVE CALCULATED DF

In [212]:
if SAVE_CALCULATED_DF:
    # dt_string = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")

    filename= SAVE_CALCULATED_DF_PATH_BASE_FOLDER + '\\' + SAVE_CALCULATED_DF_PATH_BASE_FILENAME
    print(filename)
    df.to_csv(filename, index=False)
    print('Saved', datetime.now())
    
    filename= SAVE_CALCULATED_DF_PATH_BASE_FOLDER + '\\' + SAVE_CALCULATED_SMALL_DF_PATH_BASE_FILENAME
    print(filename)
    dfsmall.to_csv(filename, index=False)
    print('Saved small', datetime.now())

D:\ICARO\Proyectos\SeleniumSundaeSwap\Calculated\sundaeswap_calculated.csv
Saved 2022-02-15 15:40:55.373628
D:\ICARO\Proyectos\SeleniumSundaeSwap\Calculated\sundaeswap_calculated_small.csv
Saved small 2022-02-15 15:40:55.470384
